In [2]:
#Load libraries 
import os
import numpy as np 
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms 
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision 
import pathlib

In [4]:
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [10]:
train_path = 'c:/Users/Ram/Desktop/Datasets'
test_path = 'c:/Users/Ram/Desktop/Dataset_Test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform = transformer),
    batch_size=32, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform = transformer),
    batch_size=32, shuffle=True
)

In [11]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [12]:
print(classes)

['Garbage classification', 'pothole_image_data']


In [13]:
class ConvNet (nn.Module): 
    def __init__ (self,num_classes=2):
        super (ConvNet, self).__init__()

        #((w-f+2P)/s) +1
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12, kernel_size=3, stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(num_features =12)
        self.relu1 = nn.ReLU()
       
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20, kernel_size=3, stride=1,padding=1)
        self.relu2 = nn.ReLU()
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32, kernel_size=3, stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(num_features =32)
        self.relu3 = nn.ReLU()
        
        
        self.fc = nn.Linear(in_features = 32*75*75,out_features = num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
            
        output = output.view(-1,32*75*75)
            
        output = self.fc(output)
        return output
            
            
            

In [14]:
model = ConvNet(num_classes = 2).to(device)

In [15]:
optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [16]:
num_epochs =10

In [17]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(train_count, test_count)

1503 1


In [19]:
best_accuracy =0.0

for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data,1)
        
        train_accuracy+= int(torch.sum(prediction==labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    model.eval()
    
    test_accuracy = 0.0
    
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _,prediction = torch.max(outputs.data,1)
        test_accuracy+= int(torch.sum(prediction==labels.data))
        
    test_accuracy = test_accuracy/test_count
    print('Epoch: '+str(epoch)+' Train Loss: '+str(int(train_loss))+ ' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    
    if test_accuracy> best_accuracy:
        torch.save(model.state_dict(),'best_checkint.model')
        best_accuracy = test_accuracy

Epoch: 0 Train Loss: 2 Train Accuracy: 0.9135063206919495 Test Accuracy: 0.0
Epoch: 1 Train Loss: 1 Train Accuracy: 0.9634065202927479 Test Accuracy: 0.0
Epoch: 2 Train Loss: 0 Train Accuracy: 0.9780439121756487 Test Accuracy: 0.0
Epoch: 3 Train Loss: 0 Train Accuracy: 0.9827012641383899 Test Accuracy: 0.0
Epoch: 4 Train Loss: 0 Train Accuracy: 0.9880239520958084 Test Accuracy: 0.0
Epoch: 5 Train Loss: 0 Train Accuracy: 0.9840319361277445 Test Accuracy: 0.0
Epoch: 6 Train Loss: 0 Train Accuracy: 0.9920159680638723 Test Accuracy: 0.0
Epoch: 7 Train Loss: 0 Train Accuracy: 0.9860279441117764 Test Accuracy: 0.0
Epoch: 8 Train Loss: 0 Train Accuracy: 0.9966733200266135 Test Accuracy: 0.0
Epoch: 9 Train Loss: 0 Train Accuracy: 0.9780439121756487 Test Accuracy: 0.0
